# Exploratory Notebook


*This notebook has been made by Martin Sejas*

### Setup - Importing packages

In [2]:
import pandas as pd
import numpy as np
import sklearn 

## Loading and Splitting Dataset 

In [5]:
#loading csv file
master = pd.read_csv("../raw_data/flight-price-training.csv")

#creating a copy for exploration
df = master.copy()

#Dropping the first column that is simply the original index of row before partitioning
df = df.drop(columns=df.columns[0])

print(df.shape)

df.head()


(240122, 11)


,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,Air_India,AI-804,Bangalore,Early_Morning,one,Night,Mumbai,Business,16.00,11,54684
1,Air_India,AI-503,Bangalore,Evening,one,Night,Hyderabad,Economy,6.25,12,13054
2,GO_FIRST,G8-426,Hyderabad,Night,one,Morning,Bangalore,Economy,10.33,34,6256
3,Indigo,6E-534,Kolkata,Evening,one,Night,Chennai,Economy,5.33,21,5280
4,Vistara,UK-863,Mumbai,Morning,two_or_more,Night,Chennai,Economy,11.17,45,8130


We can see here that we have 10 features and 1 label ('price'), we can also see a lot of our features are categorical, and encoded in an inconvenient manner, for example, the column ('stops') is in string format instead of number, we will have to see the different classes contained in this column to decide if we convert it to a number or keep it as a categorical variable. Similar treatment will be needed for other variables, such as 'departure_time' decide how we treat them. 



Lets check for any nan's, and see general info of our dataset.


In [7]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240122 entries, 0 to 240121
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   airline           240122 non-null  object 
 1   flight            240122 non-null  object 
 2   source_city       240122 non-null  object 
 3   departure_time    240122 non-null  object 
 4   stops             240122 non-null  object 
 5   arrival_time      240122 non-null  object 
 6   destination_city  240122 non-null  object 
 7   class             240122 non-null  object 
 8   duration          240122 non-null  float64
 9   days_left         240122 non-null  int64  
 10  price             240122 non-null  int64  
dtypes: float64(1), int64(2), object(8)
memory usage: 20.2+ MB


As we can see, all of the features + label match the shape of the dataframe, 240122, meaning that there are no NaN values so no imputation technique would be needed. We can proceed to check the different classes in our features.